In [8]:
import os
import sys
import math

sys.path.append('../python')
sys.path.append('../')

# sys.path.append('/home/feldheimlab/Documents/pyWholeBrain/')

from scipy.stats import poisson
from scipy.stats import nbinom
import scipy.io as sio

from preprocessing import *
from distributions_fit import *
from visualizations import *
from auditory_analysis_pipeline import data_load, get_IDs

from config import configs
from hdf5manager import hdf5manager as h5

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

config = configs()
wd = '/Users/ackmanadmin/Documents/test_dataset_auditory_pipeline/2026-02-06-0/kilosort4/'
#necessary paths
dataloc = wd

assert os.path.exists(dataloc), 'Could not find: {}'.format(dataloc)
config.dataloc = dataloc
config.parentdir = os.path.dirname(dataloc)
config.savedir = os.path.join(config.parentdir, 'results')
if not os.path.exists(config.savedir):
    os.mkdir(config.savedir)


imagedir = None
config.spikesorting = 'kilosort4'
config.class_col = 'group'
config.group = 'good'
config.probe = 'npxl'
config.rate = 30000


textoutput = os.path.join(config.savedir, 'data_processing_log.txt')
with open(textoutput, 'w') as f:
    data_raw = data_load(dataloc, spikesorting='kilosort4', class_col='group', group='good', f=f)


<bound method NDFrame.head of      cluster_id  group   ch  depth         fr
0             0   good    1    0.0   0.450578
1             1   good    0    0.0  10.324216
2             2  noise    0    0.0   0.430786
3             3   good    1    0.0   0.074942
4             4   good    0    0.0  22.231588
..          ...    ...  ...    ...        ...
678         678    mua  376  660.0  68.155559
679         679    mua  383  705.0   1.552539
680         680    mua  380  690.0   0.617779
681         681    mua  382  705.0  98.801640
682         682  noise  382  705.0  93.013506

[672 rows x 5 columns]>
group
good
[  0   1   3   4   8   9  10  13  31  34  40  47  49  51  55  58  67  74
  85  86  89  95  97 101 103 104 110 114 115 123 131 137 146 148 150 160
 161 169 173 174 181 190 191 194 195 200 201 212 213 217 220 221 232 234
 235 236 237 239 240 241 244 249 250 253 259 276 278 281 285 289 295 296
 304 324 341 352 354 359 360 361 365 372 381 387 388 397 399 401 402 404
 405 409 413 415 

In [15]:
newspeaker = data_raw.datasep[[1,2,3]]
oldspeaker = data_raw.datasep[[7,8,9]]

In [ ]:
rate = 30000
windowsz = 500/rate
verbose = True
datasets = np.array(datasep.item().get('Datasep'))/1000
print(datasets)
# n_neurons = cluster['cluster_id'].tolist()
# n_neurons = neurons['cluster_id'].tolist()
show = data_raw.IDs[:2]

# show = n_neurons




for neuron in show:
    print(neuron, data_raw.cluster.loc[data_raw.cluster['cluster_id']==neuron, 'group'])
    all_data = data_raw
    rise2 = np.array(rised)[::6]/1000
    secondm = []
    window = []
    fig, axs = plt.subplots(2, len(datasets)-1, figsize = (8,3))

    for d, dataset in enumerate(datasets[:-1]):
        start = dataset
        end = datasets[d+1]
        m_data = all_data[(all_data>=start)&(all_data< end)]
        window = np.array(rise2)[(rise2>=start)&(rise2 < end)]
        w_data = []
        if len(window)==0:
            window = np.arange(start, end+1, 1)
            window = window[window<51]
            print('default window')
        bins = np.arange(0,1+windowsz,windowsz)
        FRs = np.zeros((len(window),len(bins)-1))
        for w, win in enumerate(window):
            if w == 0:
                temp = m_data[m_data < win]-start
                axs[0][d].scatter(temp, -1*w*np.ones(temp.shape), s = .5, color = 'k')
            else:
                temp = m_data[(m_data >= window[w-1])&(m_data < win)]-window[w-1]
                axs[0][d].scatter(temp, -1*w*np.ones(temp.shape), s = .5, color = 'k')
            w_data.extend(temp)
            FRs[w], _ = np.histogram(temp, bins=bins)
        if d==0:
            poislambda = np.mean(FRs)
            poisdisper = np.std(FRs)**2
            probs = 1 - poisson.cdf(FRs/poisdisper, poislambda/poisdisper)
            if verbose:
                print('Lambda: {0}, dipsersion: {1}'.format(poislambda, poisdisper))

        else:   
            probs = 1 - poisson.cdf(FRs/poisdisper, poislambda/poisdisper)
            probsmean = 1 - poisson.cdf(np.mean(FRs,0)/poisdisper, poislambda/poisdisper)

            meanprobs = np.mean(np.squeeze(np.sum([probs< 0.01],1))/probs.shape[0])
            stdprobs = np.std(np.squeeze(np.sum([probs< 0.01],1))/probs.shape[0])

            sigprobs = 1-norm.cdf(np.squeeze(np.sum([probs < 0.01],1))/probs.shape[0],
                          meanprobs , stdprobs)

        axs[1][d].errorbar(bins[:-1]+windowsz/2, np.mean(FRs/(windowsz),0),
                           yerr=[np.zeros(len(bins)-1),np.std(FRs/(windowsz),0)/FRs.shape[0]], 
                        fmt = 'o', color='grey')
        secondm.append(axs[1][d].twinx())
        secondm[d].plot(bins[:-1]+windowsz/2, np.squeeze(np.sum([probs < 0.01],1))/probs.shape[0], color='r')

#         wave = fig.add_axes([0.25,0.35,0.1,0.1])
#         tmax = np.max(templates[neuron,:,cluster.loc[neuron, 'ch']])
#         if tmax >0.005:
#             wave.plot(-1*templates[neuron,:,cluster.loc[neuron, 'ch']], linewidth=1, color='k')
#         else:
#             wave.plot(templates[neuron,:,cluster.loc[neuron, 'ch']], linewidth=1, color='k')
#         wave.axis('off')
        axs[0][d].set_xticks([0,0.5,1,1.5,2])
        axs[0][d].set_xticklabels([])
        axs[1][d].set_xticks([0,0.5,1,1.5,2])
        axs[0][d].set_xlim(0,1)
        axs[0][d].set_yticklabels([])
    clims = [0,0]
    plims = [0,0]
    for d, dataset in enumerate(datasets[:-1]):
        lims = axs[1][d].get_ylim()
        if lims[1] > clims[1]:
            clims[1] = lims[1]
        lims = secondm[d].get_ylim()
        if lims[1] > plims[1]:
            plims[1] = lims [1]
    for d, dataset in enumerate(datasets[:-1]):
        axs[1][d].set_xlim(0,1)
        axs[1][d].set_ylim(clims)
        secondm[d].set_ylim(plims)
#         if d==1:
#             axs[1][d].fill_betweenx((0,clims[1]), 1, color='blue', alpha=(d)/(2),zorder=0)

    axs[1][1].set_xlabel('time (sec)')
    axs[1][0].set_ylabel('FR (spikes per sec)', color='grey')
    axs[0][0].set_ylabel('Trial')
    axs[0][0].set_title('Spont.')
#     axs[0][1].set_title('Aud. stim')
    axs[0][1].set_title('+ 30% laser')

#     axs[0][3].set_title('+ 5% laser')
#     axs[0][5].set_title('+ 50% l')
#     axs[0][6].set_title('+ 75% l')
#     axs[0][7].set_title('+ 100% l')
    for a in np.arange(1):
        axs[1][a+1].set_yticklabels([])
        secondm[a].set_yticklabels([])


    secondm[len(datasets)-2].set_ylabel('Precent of trials w/ sig FR', color='r')
#     plt.savefig('{0}_cluster_aud_resp.png'.format(neuron))
    plt.show()